# Prompt selection and testing

This notebook chooses the most appropriate prompt and prompt structure for the OCR correction. 

In [17]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, compare_request_configurations
from evaluate import load
from evaluation_funcs import evaluate_correction_performance, evaluate_correction_performance_folders, get_metric_error_reduction
from helper_functions import files_to_df_func, files_to_df_core_func
import seaborn as sns
import matplotlib.pyplot as plt

dev_data_folder = 'data/dev_data'
dev_transcripts = os.path.join(dev_data_folder, 'dev_data_transcript')
dev_raw_ocr_folder =  os.path.join(dev_data_folder,'dev_raw_ocr' )
dev_system_message_folder = os.path.join(dev_data_folder,'dev_system_message_variants' )

#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv(os.path.join(dev_data_folder,'dev_data_raw.csv'))


#for saving data to be used in the analysis
if not os.path.exists('data/analysis'):
    os.makedirs('data/analysis')

# Explore different system prompts

This explores a range of system prompt to find the one that appears to work the best, we use gpt4 as the baseline model.

Although there is no comparison with all models we do test gpt3.5 gpt4, clause haiku and claude opus, in addition we put the prompt in the system message and the prompt after the text becuase this may affect the quality of the result. When the prompt is after the text the response has "nosm_" (no system message) appended to the file name

In [2]:
#Create a modular set of system messages that can be combined in different ways
basic_prompt = "Please recover the text from the corrupted OCR."
expertise_prompt = "You are an expert in post-OCR correction of documents."
recover_prompt = "Using the context available from the text please recover the most likely original text from the corrupted OCR."
publication_context_prompt = "The text is from an english newspaper in the 1800's."
text_context_prompt = "The text may be an advert or article and may be missing the beggining or end."
additional_instructions_prompt = "Do not add any text, commentary, or lead in sentences beyond the recovered text. Do not add a title, or any introductions."

#combine all the message parts into a variety of system messages, a tuple is used where 0 is the name of the message and 1 is the message itself
#N.B. This is not and exhaustive combination as that would be very expensive and likley not yield significantly better results
system_messages_list = [
('basic_prompt', basic_prompt),
('expert_basic_prompt', expertise_prompt + ' '+ basic_prompt),
('expert_recover_prompt', expertise_prompt + ' '+ recover_prompt),
('expert_recover_publication_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt),
('expert_recover_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + text_context_prompt),
('expert_recover_publication_text_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt),
('expert_recover_instructions_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + additional_instructions_prompt),
('full_context_prompt', expertise_prompt + ' '+ recover_prompt + ' ' + publication_context_prompt + ' ' + text_context_prompt+ ' ' + additional_instructions_prompt)
]

The below function is used to make the creation of the config dictionaries for the test more compact and increase readability

In [3]:
def create_message_test_configs(system_messages_list, get_response_func, engine):
    message_test_configs = []
    for iter_system_message in system_messages_list:
        message_test_configs.append(
            create_config_dict_func(
                get_response_func=get_response_func,
                rate_limiter=RateLimiter(50000),
                engine=engine,
                system_message_template=iter_system_message[1],
                prompt_template="{content_html}",
                additional_args={'response_name': iter_system_message[0]}
            )
        )
        message_test_configs.append(
            create_config_dict_func(
                get_response_func=get_response_func,
                rate_limiter=RateLimiter(50000),
                engine=engine,
                system_message_template="",
                prompt_template="{content_html}" + f"""\n\n""" + iter_system_message[1],
                additional_args={'response_name': "nosm_"+iter_system_message[0]}
            )
        )
    return message_test_configs

## Create configs and run tests

In [4]:
#gpt configs
gpt3_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_openai, "gpt-3.5-turbo")
gpt4_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_openai, 'gpt-4-turbo-preview')

#claude configs
haiku_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_anthropic, "claude-3-haiku-20240307")
opus_prompt_testing_configs = create_message_test_configs(system_messages_list, get_response_anthropic, "claude-3-opus-20240229")

#run the experiment on all the prompt configs and save to the folder
compare_request_configurations(dev_data_df, 
                               gpt3_prompt_testing_configs + gpt4_prompt_testing_configs + haiku_prompt_testing_configs + opus_prompt_testing_configs,
                               folder_path = dev_system_message_folder )

In [35]:

prompt_test_df = []
for folder in os.listdir(dev_system_message_folder):
    df = files_to_df_core_func(dev_system_message_folder)
    df['folder'] = folder
    prompt_test_df.append(df)

prompt_test_df = pd.concat(prompt_test_df, ignore_index=True)

# Evaluate system prompt tests

We evaluate the system prompts below to see if thre is any significant difference between the prompts


In [37]:
wer = load("wer")
cer = load("cer")

In [38]:
raw_dev_ocr_scores = evaluate_correction_performance(dev_raw_ocr_folder , dev_transcripts, wer, cer, 'raw_ocr')

In [39]:
corrected_folder = dev_system_message_folder

performance_eval = evaluate_correction_performance_folders(corrected_folder, dev_transcripts, wer, cer)

performance_eval =  performance_eval.loc[(performance_eval['File Name']!='slug_ar02501_periodical_pc_issue_tec_06121884_page_number_25.txt'),:]

performance_eval['type'] = performance_eval['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")

performance_eval['model'] = performance_eval['type'].str.split('_').str[-1]

In [8]:
test = get_metric_error_reduction(performance_eval, raw_dev_ocr_scores)

In [12]:
test.groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER            CER  \
                                                     mean    50%    mean   
type                                                                       
expert_recover_text_prompt_haiku                    -3.36  13.64 -296.74   
nosm_expert_recover_publication_text_prompt_haiku    4.55  18.18 -294.91   
expert_recover_publication_text_prompt_haiku        17.73  35.45 -209.70   
nosm_expert_recover_text_prompt_haiku                1.70  11.65 -297.54   
expert_recover_prompt_haiku                         39.84  48.11  -90.64   
...                                                   ...    ...     ...   
nosm_full_context_prompt_gpt-4-turbo-preview        73.21  77.19   54.38   
expert_recover_instructions_prompt_claude-3-opu...  73.67  80.46   46.82   
nosm_expert_basic_prompt_gpt-4-turbo-preview        73.61  77.59   55.55   
nosm_expert_recover_prompt_gpt-4-turbo-preview      72.03  75.86   49.27   
nosm_full_context_prompt_claude-3-opus-20240229     73.42  80.46   58.95   

                                                           lev_dist          
                                                       50%     mean     50%  
type                                                                         
expert_recover_text_prompt_haiku                   -197.56  -282.00 -177.78  
nosm_expert_recover_publication_text_prompt_haiku  -152.94  -279.84 -144.31  
expert_recover_publication_text_prompt_haiku       -115.22  -198.48 -110.20  
nosm_expert_recover_text_prompt_haiku              -107.24  -280.89 -102.16  
expert_recover_prompt_haiku                         -39.82   -85.44  -36.21  
...                                                    ...      ...     ...  
nosm_full_context_prompt_gpt-4-turbo-preview         63.78    52.09   62.23  
expert_recover_instructions_prompt_claude-3-opu...   68.01    45.19   62.87  
nosm_expert_basic_prompt_gpt-4-turbo-preview         62.80    53.41   63.64  
nosm_expert_recover_prompt_gpt-4-turbo-preview       65.55    47.03   63.79  
nosm_full_context_prompt_claude-3-opus-20240229      68.78    57.84   66.36  

[64 rows x 6 columns]

In [11]:
test.loc[test['type'].str.contains('opus')].groupby('type').describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER           CER  \
                                                     mean    50%   mean   
type                                                                      
nosm_expert_recover_publication_text_prompt_cla...  56.55  74.70 -29.86   
nosm_expert_recover_publication_prompt_claude-3...  71.64  75.47  44.57   
nosm_basic_prompt_claude-3-opus-20240229            70.73  76.02  47.31   
expert_recover_publication_prompt_claude-3-opus...  71.07  77.36  41.74   
expert_recover_text_prompt_claude-3-opus-20240229   72.19  77.36  44.01   
nosm_expert_recover_prompt_claude-3-opus-20240229   72.85  76.02  49.35   
basic_prompt_claude-3-opus-20240229                 72.25  77.36  41.81   
expert_recover_prompt_claude-3-opus-20240229        63.66  78.30   4.86   
expert_recover_publication_text_prompt_claude-3...  70.79  77.36  42.16   
full_context_prompt_claude-3-opus-20240229          68.79  77.39  25.49   
nosm_expert_recover_text_prompt_claude-3-opus-2...  71.35  75.86  50.45   
nosm_expert_basic_prompt_claude-3-opus-20240229     73.73  77.59  56.30   
expert_basic_prompt_claude-3-opus-20240229          75.92  79.25  59.19   
nosm_expert_recover_instructions_prompt_claude-...  73.95  80.46  55.57   
expert_recover_instructions_prompt_claude-3-opu...  73.67  80.46  46.82   
nosm_full_context_prompt_claude-3-opus-20240229     73.42  80.46  58.95   

                                                          lev_dist         
                                                      50%     mean    50%  
type                                                                       
nosm_expert_recover_publication_text_prompt_cla...  43.62   -29.53  45.00  
nosm_expert_recover_publication_prompt_claude-3...  49.56    43.02  48.07  
nosm_basic_prompt_claude-3-opus-20240229            59.73    45.54  49.76  
expert_recover_publication_prompt_claude-3-opus...  55.14    39.58  52.66  
expert_recover_text_prompt_claude-3-opus-20240229   56.22    42.49  53.72  
nosm_expert_recover_prompt_claude-3-opus-20240229   58.54    47.96  53.88  
basic_prompt_claude-3-opus-20240229                 58.38    40.48  55.60  
expert_recover_prompt_claude-3-opus-20240229        63.23     5.51  55.60  
expert_recover_publication_text_prompt_claude-3...  60.00    40.97  55.60  
full_context_prompt_claude-3-opus-20240229          55.41    25.91  56.00  
nosm_expert_recover_text_prompt_claude-3-opus-2...  58.71    49.13  58.31  
nosm_expert_basic_prompt_claude-3-opus-20240229     64.86    54.26  58.99  
expert_basic_prompt_claude-3-opus-20240229          64.39    56.97  61.79  
nosm_expert_recover_instructions_prompt_claude-...  69.66    53.61  61.94  
expert_recover_instructions_prompt_claude-3-opu...  68.01    45.19  62.87  
nosm_full_context_prompt_claude-3-opus-20240229     68.78    57.84  66.36

In [40]:
performance_eval2 = performance_eval.copy()
performance_eval2['type'] = performance_eval2['type'].str.replace("claude-3-haiku-20240307", "haiku").replace("gpt-3.5-turbo", "gpt-3.5")
performance_eval2['model'] = performance_eval2['type'].str.split('_').str[-1]
#performance_eval2 = performance_eval2.loc[performance_eval2['model'].str.contains('gpt-4')]
performance_eval2.drop(columns = 'File Name').groupby(['type', 'model']).describe().filter(regex = '50|mean').round(2).sort_values(('lev_dist', '50%'))

WER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.23   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.23   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.24   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.24   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.25   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.57   
expert_recover_publication_text_prompt_haiku       haiku                   0.71   
nosm_expert_recover_text_prompt_haiku              haiku                   0.81   
expert_recover_text_prompt_haiku                   haiku                   0.83   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.81   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.15   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.12   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.16   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.11   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.51   
expert_recover_publication_text_prompt_haiku       haiku                   0.51   
nosm_expert_recover_text_prompt_haiku              haiku                   0.73   
expert_recover_text_prompt_haiku                   haiku                   0.90   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.68   

                                                                            CER  \
                                                                           mean   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.13   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.13   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.14   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.14   
nosm_full_context_prompt_claude-3-opus-20240229    claude-3-opus-20240229  0.15   
...                                                                         ...   
nosm_expert_recover_publication_prompt_haiku       haiku                   0.44   
expert_recover_publication_text_prompt_haiku       haiku                   0.58   
nosm_expert_recover_text_prompt_haiku              haiku                   0.65   
expert_recover_text_prompt_haiku                   haiku                   0.68   
nosm_expert_recover_publication_text_prompt_haiku  haiku                   0.66   

                                                                                 \
                                                                            50%   
type                                               model                          
full_context_prompt_gpt-4-turbo-preview            gpt-4-turbo-preview     0.06   
nosm_expert_recover_instructions_prompt_gpt-4-t... gpt-4-turbo-preview     0.06   
expert_recover_instructions_prompt_claude-3-opu... claude-3-opus-20240229  0.05   
nosm_full_context_prompt_gpt-4-turbo-preview       gpt-4-turbo-preview     0.07   
nosm_f

In [46]:
result = performance_eval2.drop(columns='File Name') \
    .groupby(['model', 'type'])['lev_dist'] \
    .median() \
    .reset_index() \
    .sort_values('lev_dist') \
    .groupby('model') \
    .head(2) \
    .set_index(['model', 'type']).sort_values('model')

result

lev_dist
model                  type                                                        
claude-3-opus-20240229 expert_recover_instructions_prompt_claude-3-opu...      75.0
                       nosm_full_context_prompt_claude-3-opus-20240229         76.0
gpt-3.5-turbo          expert_recover_publication_text_prompt_gpt-3.5-...     111.0
                       expert_recover_publication_prompt_gpt-3.5-turbo        111.0
gpt-4-turbo-preview    full_context_prompt_gpt-4-turbo-preview                 72.0
                       nosm_expert_recover_instructions_prompt_gpt-4-t...      75.0
haiku                  expert_recover_instructions_prompt_haiku               110.0
                       nosm_expert_recover_instructions_prompt_haiku          122.0

## Conclusions of the prompt test

It appears that the placing the prompt after the text instead of using the system prompt gives the best results. However, the prompts did give significantlty different performance.  I think that perhaps using the `full_context_prompt` and the `expert_recover_publication_prompt` with no system message and the prompt after the text may be the best option. This will require twice as much compute as I was planning to use.

# Evaluating models on the test set

Having identified two different prompts and that the prompts appear to work better when places after the text we can now compare the different models


The below code creates the basic configuration dictionaries for each model and then fills in the with the two different prompt messages creating a single list of all basic prompt/model configurations. It then calls all the LLM's and saves the results.
This works in series so takes a while.